In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv
from collections import Counter
import docker
import re

c:\Users\janva\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
client = docker.from_env()
container = client.containers.get('splash')
#start container
container.start()

In [44]:
def get_soup(url):
    t = 0
    if url.startswith("https://www.amazon.com/dp/") or url.startswith("https://www.amazon.com/s?i"):
        time_wait = 20
    else:
        time_wait = 2
    while True:
        r = requests.get("http://localhost:8050/render.html", params={"url": url,'wait': time_wait, 'viewport': '320x480', 'images': 0, 'resource_timeout': 10})
        soup = BeautifulSoup(r.text, "html.parser")
        if soup.text.find('GlobalTimeoutError') != -1:
            container.restart()
            time.sleep(31)
            print(f"time sleep try {t}")
            time_wait += 4
            if t == 2:
                print(f"too many timeouts (2) for url: {url}")
                break
            t += 1
        else:
            print(f"page successfully loaded in time {time_wait}: {url}")
            break       
    return soup

In [4]:
def get_link_asin(link):
    link_asin = {}
    splitted_link = link['href'].split('/')
    name = splitted_link[1]
    link_asin['link'] = name
    asin = splitted_link[3]
    link_asin['asin'] = asin
    #You can create the link to the review page with the name and asin
    link_asin['full_link'] = "https://www.amazon.com/"+str(name)+"/product-reviews/"+str(asin)+"/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber="
    return link_asin

In [5]:
with open(f"../../gen/input/amazon_links.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["name", "asin", "product_link", "page_link"])


In [6]:
all_product_links = []
i = 1
with open(f"../../gen/input/amazon_links.csv", "a", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    while i <= 10:
        #page link filtered on Ios, android and windows (to avoid including non-smartphone phones) and including out-of-stock
        page_link = f"https://www.amazon.com/s?i=hpc-intl-ship&bbn=16225010011&rh=n%3A16225010011%2Cn%3A3777891&page={i}&qid=1709220798&ref=sr_pg_{i}"
        soup = get_soup(page_link)
        print(f"currently looking at page {i}")
        #Block usually after about 100 pages
        if soup.title.text=="Sorry! Something went wrong!":
            print("Blocked")
            break
        #If not block, gather asin and link
        else:
            links = soup.find_all(class_="a-link-normal s-no-outline")
        for link in links:
            if link['href'].find('/dp/') != -1 and not link['href'].startswith('/dp/'):
                link_asin = get_link_asin(link)
                link_asin['page_link'] = page_link
                all_product_links.append(link_asin)
                writer.writerow(list(link_asin.values()))
        #Try to find next page button. If absent, break
        try:
            soup.find('span', class_ = "s-pagination-item s-pagination-next s-pagination-disabled").text
            print('last page')
            break
        except:
            pass
        print(len(all_product_links))
        i += 1

page successfully loaded in time 20: https://www.amazon.com/s?i=hpc-intl-ship&bbn=16225010011&rh=n%3A16225010011%2Cn%3A3777891&page=1&qid=1709220798&ref=sr_pg_1
currently looking at page 1
12
page successfully loaded in time 20: https://www.amazon.com/s?i=hpc-intl-ship&bbn=16225010011&rh=n%3A16225010011%2Cn%3A3777891&page=2&qid=1709220798&ref=sr_pg_2
currently looking at page 2
24
page successfully loaded in time 20: https://www.amazon.com/s?i=hpc-intl-ship&bbn=16225010011&rh=n%3A16225010011%2Cn%3A3777891&page=3&qid=1709220798&ref=sr_pg_3
currently looking at page 3
36
page successfully loaded in time 20: https://www.amazon.com/s?i=hpc-intl-ship&bbn=16225010011&rh=n%3A16225010011%2Cn%3A3777891&page=4&qid=1709220798&ref=sr_pg_4
currently looking at page 4
48
page successfully loaded in time 20: https://www.amazon.com/s?i=hpc-intl-ship&bbn=16225010011&rh=n%3A16225010011%2Cn%3A3777891&page=5&qid=1709220798&ref=sr_pg_5
currently looking at page 5
60
page successfully loaded in time 20: htt

In [7]:
with open(f"../../data/amazon.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["asin_url", "brand", "model", "link", "product_title",
             "name", "rating", "review_title", "country", "date",
             "specs", "first_spec", "review", "helpful", "picture",
            "verified", "profile", "video"])

In [ ]:
#close container


In [53]:
#This function retrieves all the review info from a page
def get_reviews_from_page(product_info, start=1, end = 500):
    print(f"succesfully loaded {product_info['title']}")
    with open(f"../../data/amazon.csv", "a", encoding = "UTF-8", newline = '') as csv_file:
        writer = csv.writer(csv_file, delimiter = ";")
        for i in range(start,end):
            missing_page = []
            soup_reviews = get_soup((product_info['link']+str(i)).replace("ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=", f"ref=cm_cr_getr_d_paging_btm_next_{i}?pageNumber="))
            reviews = soup_reviews.find_all('div', {"data-hook":"review"})
            if len(reviews) == 0:
                print(f'Could not find any reviews at page {i}')
                break
            else:
                pass

            #check if chaptcha
            if soup.find('h4').text == 'Enter the characters you see below':
                print('chaptcha check')
                container.stop()
                raise Exception('chaptcha')

            for review in reviews:
                review_info = {}
                review_info = {
                'product_url' : product_info['link']+str(i),
                'brand' : product_info['brand'],
                'model' : product_info['model'],
                'link' : product_info['link']+str(i),
                'product_title' : product_info['title']
                }
                try:
                    review_info['name'] = review.find('span', class_="a-profile-name").text
                except:
                    review_info['name'] = None
                try:
                    review_info['star_rating'] = float(review.find('i', {"data-hook":"review-star-rating"}).text.replace(' out of 5 stars',''))
                except:
                    #foreign countries
                    try:
                        review_info['star_rating'] = float(review.find('i', {"data-hook":"cmps-review-star-rating"}).text.replace(' out of 5 stars',''))
                    except:
                        review_info['star_rating'] = None
                try:
                    review_info['title'] = review.find('a', {"data-hook":"review-title"}).text.strip()
                except:
                    try:
                        #foreign countries
                        review_info['title'] = review.find('span',{"data-hook":"review-title"}).text.strip()
                    except:
                        review_info['title'] = None
                try:
                    review_info['country'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[0]
                except:
                    review_info['country'] = None
                try:
                    review_info['date'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[2].strip()
                except:
                    review_info['date'] = None
                attr = []
                try:
                    attributes = str(reviews[0].find('a', {"data-hook":"format-strip"}))
                    for item in re.findall(r'[>](.+?)[<]', attributes):
                        attr.append(item.replace('</i>',''))
                    review_info['info'] = attr
                except:
                    review_info['info'] = None
                try:
                    for at in attr:
                        if at.find("Color:") != -1:
                            color = at.replace("Color: ", "")
                    review_info['color'] = color
                except:
                    review_info['color'] = None
                try:
                    review_info['review'] = review.find('span', {"data-hook":"review-body"}).text.strip()
                except:
                    review_info['review'] = None
                try:
                    helpful = review.find('span', {"data-hook" : "helpful-vote-statement"}).text.strip()
                    if helpful == "One person found this helpful":
                        review_info['helpful'] = 1
                    elif "found this helpful" in helpful:
                        review_info['helpful'] = helpful.split()[0]
                    else:
                        review_info['helpful'] = helpful
                except:
                    review_info['helpful'] = None
                try:
                    #get links for consumer pictures
                    if review.find('div', class_="review-image-tile-section") is None:
                        review_info['picture'] = 'no'
                    else:
                        pics = review.find('div', class_ = "a-section a-spacing-top-mini cr-lightbox-image-thumbnails").find_all('img')
                        review_info['picture'] = [img['src'] for img in pics]
                except:
                    review_info['picture'] = None

                try:
                    if review.find('span', {"data-hook":"avp-badge"}) is None:
                        review_info['verified'] = 'no'
                    else:
                        review_info['verified'] = 'yes'
                except:
                    review_info['verified'] = None
                    
                try:
                    pf_link = review.find('div', class_="a-profile-avatar").find('img')['data-src']
                    #check if it is a default image
                    if pf_link.find('default') != -1:
                        pf_link = None
                    else:
                        #saving picture
                        try:
                            r = requests.get(pf_link)
                            with open(f"../../gen/output/photos/"+pf_link.split('/')[-1], "wb") as f:
                                f.write(r.content)
                        except:
                            print('error while downloading profile picture')            
                    review_info['profile_picture'] = pf_link

                except:
                    review_info['profile_picture'] = None
                    
                try:
                    video = review.find('div', class_="vse-airy-container vse-player-container")
                    if video == None:
                        review_info['video'] = 'no'
                    else:
                        review_info['video'] = 'yes'
                except:
                    review_info['video'] = None

                all_reviews.append(review_info)
                missing_values = [key for key,value in review_info.items() if value == None]
                missing_page += missing_values
                writer.writerow(list(review_info.values()))
            print(len(all_reviews))
            if len(all_reviews) % 500 == 0:
                print('restart container')
                container.restart()
                time.sleep(15)
            #missing values checker
            if len(missing_page) > 0:
                print(f"the following items were missing: {dict(Counter(missing_page))}")

            if not soup_reviews.find('li', class_ = "a-disabled a-last"):
                pass
            else:
                print(f"page {i} was the last page")
                break

In [14]:
if 1010 % 100 == 0:
    print('yes')

In [10]:
#load csv using pandas
import pandas as pd
import re
all_reviews = []
#no headers
links_data = pd.read_csv(f"../../gen/input/amazon_links.csv", delimiter = ";")
links_data

,name,asin,product_link,page_link
0,GuruNanda-Essential-Vitamins-Mouthwash-Supports,B0BVFXGMLW,https://www.amazon.com/GuruNanda-Essential-Vit...,https://www.amazon.com/s?i=hpc-intl-ship&bbn=1...
1,Mighty-Patch-Hydrocolloid-Absorbing-count,B074PVTPBW,https://www.amazon.com/Mighty-Patch-Hydrocollo...,https://www.amazon.com/s?i=hpc-intl-ship&bbn=1...
2,PanOxyl-Foaming-Peroxide-Strength-Antimicrobial,B081KL2QYJ,https://www.amazon.com/PanOxyl-Foaming-Peroxid...,https://www.amazon.com/s?i=hpc-intl-ship&bbn=1...
3,Neutrogena-Cleansing-Towelettes-Waterproof-Alc...,B00U2VQZDS,https://www.amazon.com/Neutrogena-Cleansing-To...,https://www.amazon.com/s?i=hpc-intl-ship&bbn=1...
4,CeraVe-Moisturizing-Cream-Daily-Moisturizer,B00TTD9BRC,https://www.amazon.com/CeraVe-Moisturizing-Cre...,https://www.amazon.com/s?i=hpc-intl-ship&bbn=1...
...,...,...,...,...
115,Philips-QP230-50-Oneblade-Replacement,B0743KVCJC,https://www.amazon.com/Philips-QP230-50-Onebla...,https://www.amazon.com/s?i=hpc-intl-ship&bbn=1...
116,Bio-Oil-4-2oz-Multiuse-Skincare-Oil,B004AI97MA,https://www.amazon.com/Bio-Oil-4-2oz-Multiuse-...,https://www.amazon.com/s?i=hpc-intl-ship&bbn=1...
117,Colgate-Toothbrush-Toothbrushes-Ergonomic-Circ...,B00CC6XSRC,https://www.amazon.com/Colgate-Toothbrush-Toot...,https://www.amazon.com/s?i=hpc-intl-ship&bbn=1...
118,Vicks-VapoShower-Aromatherapy-Eucalyptus-Essen...,B08QTZBK5Y,https://www.amazon.com/Vicks-VapoShower-Aromat...,https://www.amazon.com/s?i=hpc-intl-ship&bbn=1...


In [85]:
all_reviews = []

In [54]:
#view links so far from amazon.csv
data_so_far = pd.read_csv(f"../../data/amazon.csv", delimiter = ";")
links_so_far = data_so_far['asin_url'].unique()
#remove last digits in links_so_far
links_so_far = [re.sub(r'\d+$', '', link) for link in links_so_far]
links_left = [links for links in links_data['product_link'] if links not in links_so_far]

l = 1
for link in links_left:
    print(f"{l} / {len(links_left)}")
    product_info = {
    'product_url' : "https://www.amazon.com/dp/"+link.split('/')[-2],
    'brand' : None,
    'model' : None,
    'link' : link,
    'title' : link.split('/')[3],
    'product_title' : None
    }
    get_reviews_from_page(product_info, 1, 15)
    l += 1
container.stop()

1 / 36
succesfully loaded Davids-Hydroxyapatite-Toothpaste-Sensitive-Peppermint
page successfully loaded in time 2: https://www.amazon.com/Davids-Hydroxyapatite-Toothpaste-Sensitive-Peppermint/product-reviews/B09JHSVM58/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
chaptcha check


Exception: chaptcha

In [47]:
soup = get_soup("https://www.amazon.com/dp/B07Y5PCHZ7")

page successfully loaded in time 20: https://www.amazon.com/dp/B07Y5PCHZ7


In [48]:
soup

<!DOCTYPE html>
<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]--><!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]--><!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.com</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
        ue_mid = "ATVPDKIKX0DER",
   

In [52]:
#get soup header 
if soup.find('h4').text == 'Enter the characters you see below':
    

True

In [76]:
link = "https://www.amazon.com/Revlon-Kiss-Exfoliating-Balm-Sugar/product-reviews/B0759HSBQC/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber="
i = 9
soup_reviews = get_soup((link+str(i)).replace("ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=", f"ref=cm_cr_getr_d_paging_btm_next_{i}?pageNumber="))
reviews = soup_reviews.find_all('div', {"data-hook":"review"})

page successfully loaded in time 2: https://www.amazon.com/Revlon-Kiss-Exfoliating-Balm-Sugar/product-reviews/B0759HSBQC/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=9


# testing

In [77]:
#find image
pics = reviews[0].find('div', class_ = "a-section a-spacing-top-mini cr-lightbox-image-thumbnails").find_all('img')
[img['src'] for img in pics]

['https://m.media-amazon.com/images/I/61Mvtq4n9oL._SY88.jpg']

In [60]:
#to dataframe
data = pd.DataFrame(all_reviews)
#filter out the ones with no profile picture
profile_pictures = data[data['profile_picture'].notna()]['profile_picture']




'https://images-na.ssl-images-amazon.com/images/S/amazon-avatars-global/2e87994e-58dc-4526-9e8f-bb8492f614b1._CR0,0,363,363_SX48_.jpg'

In [62]:
profile_pictures.iloc[0].split('/')[-1]

'2e87994e-58dc-4526-9e8f-bb8492f614b1._CR0,0,363,363_SX48_.jpg'

In [63]:
#download the profile pictures
for i in range(len(profile_pictures)):
    r = requests.get(profile_pictures.iloc[i])
    with open(f"../../gen/output/photos/"+profile_pictures.iloc[i].split('/')[-1], "wb") as f:
        f.write(r.content)
    time.sleep(1)

In [87]:
pf_link = "https://images-na.ssl-images-amazon.com/images/S/amazon-avatars-global/4d0e1bc9-8ce1-41e4-bc10-15a8df734c56._CR0,0,423,423_SX48_.jpg"
r = requests.get(pf_link)
with open(f"../../gen/output/photos/"+pf_link.split('/')[-1], "wb") as f:
    f.write(r.content)